In [1]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import torch.optim as optim

import time

from tqdm import tqdm
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

/Users/junginseo/opt/anaconda3/envs/dl_env/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [31]:
class FaceDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.label_list = pd.read_csv(csv_file)
        self.label_list['name_en'] = self.label_list['name'].astype('category').cat.codes
        self.label_list['file_name'] = self.label_list['file_name'].apply(lambda x : x.replace(" ", ""))
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.label_list)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.label_list.iloc[idx]['file_name'])
        image = io.imread(img_name)
        label = self.label_list.iloc[idx]['name_en']
        if self.transform:
            image = self.transform(image)
        return image, label

In [32]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

In [33]:
train_dataset = FaceDataset(csv_file='data/kid_f_train.csv',
                                    root_dir='data/HQ_512x512/HQ_512x512/', 
                                    transform= transform)
test_dataset = FaceDataset(csv_file='data/kid_f_test.csv',
                            root_dir='data/test_final_with_degrad/test/', 
                            transform= transform)

In [34]:
train_loader = torch.utils.data.DataLoader(train_dataset,          # 원래는 별도의 Validation Set의 root 경로를 지정해야 한다. 
                                          batch_size=20,         # 배치사이즈 지정
                                          shuffle=True)           # shuffle 여부 지정
test_loader = torch.utils.data.DataLoader(test_dataset,          # 원래는 별도의 Validation Set의 root 경로를 지정해야 한다. 
                                          batch_size=20,         # 배치사이즈 지정
                                          shuffle=True)           # shuffle 여부 지정

In [6]:
#
import torch
import torch.nn as nn
class VGG_11(nn.Module):
    def __init__(self, num_classes: int = 300, init_weights: bool = True):
        super(VGG_11, self).__init__()
        self.convnet = nn.Sequential(
            # Input Channel (RGB: 3)
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # 224 -> 112
            
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # 112 -> 56
            
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # 56 -> 28

            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # 28 -> 14

            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # 14 -> 7
        )

        self.fclayer = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes),
            # nn.Softmax(dim=1), # Loss인 Cross Entropy Loss 에서 softmax를 포함한다.
        )
    
    def forward(self, x:torch.Tensor):
        x = self.convnet(x)
        x = torch.flatten(x, 1)
        x = self.fclayer(x)
        return x

In [13]:
#
import torch
import torch.nn as nn
class VGG_19(nn.Module):
    def __init__(self, num_classes: int = 300, init_weights: bool = True):
        super(VGG_19, self).__init__()
        self.convnet = nn.Sequential(
            # Input Channel (RGB: 3)
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # 224 -> 112
            
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # 112 -> 56
            
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),

            nn.MaxPool2d(kernel_size=2, stride=2), # 56 -> 28

            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # 28 -> 14

            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # 14 -> 7
        )

        self.fclayer = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes),
            # nn.Softmax(dim=1), # Loss인 Cross Entropy Loss 에서 softmax를 포함한다.
        )
    
    def forward(self, x:torch.Tensor):
        x = self.convnet(x)
        x = torch.flatten(x, 1)
        x = self.fclayer(x)
        return x

In [14]:
device = 'mps'
vgg19 = VGG_19()
vgg19 = vgg19.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg19.parameters(),lr=0.0001)

start_time = time.time()
for epoch in tqdm(range(1000)):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # print(inputs.shape)  
        outputs= vgg19(inputs)

        # print(outputs.shape)
        # print(labels.shape)
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 50 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 50))
            running_loss = 0.0

print(time.time()-start_time)
print('Finished Training')

  0%|          | 0/10 [00:00<?, ?it/s]

[1,    50] loss: 5.698
[1,   100] loss: 5.691
[1,   150] loss: 5.682
[1,   200] loss: 5.674
[1,   250] loss: 5.666


 10%|█         | 1/10 [05:33<50:04, 333.82s/it]

[2,    50] loss: 5.653
[2,   100] loss: 5.645
[2,   150] loss: 5.637
[2,   200] loss: 5.630
[2,   250] loss: 5.621


 20%|██        | 2/10 [10:59<43:52, 329.09s/it]

...

[9,    50] loss: 5.342
[9,   100] loss: 5.337
[9,   150] loss: 5.327
[9,   200] loss: 5.317
[9,   250] loss: 5.311


 90%|█████████ | 9/10 [50:06<05:33, 333.22s/it]

[10,    50] loss: 5.298
[10,   100] loss: 5.293
[10,   150] loss: 5.285
[10,   200] loss: 5.275
[10,   250] loss: 5.266


100%|██████████| 10/10 [55:09<00:00, 330.97s/it]

3309.6829240322113
Finished Training


In [35]:
vgg19.eval()    # 평가시에는 dropout이 OFF 된다.
correct = 0
for data, target in test_loader:
    data = data.to(device)
    target = target.to(device)
    output = vgg19(data)
    prediction = output.data.max(1)[1]
    correct += prediction.eq(target.data).sum()
print('Test set Accuracy : {:.2f}%'.format(100. * correct / len(test_loader.dataset)))

Test set Accuracy : 0.00%
